In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os
import re
from itertools import compress
import time
import random
import math

# Cross validation

In [3]:
df = pd.read_json("../data/debatepedia/debatepedia-preprocessed.json", orient = "records")
df

,argument_id,content,topic,unitType
0,0,Not passing $700b bailout risks sending econom...,$700 billion US economic bailout,conclusion
1,0,U.S. Treasury Secretary Henry Paulson summariz...,$700 billion US economic bailout,premise
2,1,$700b bailout helps avoid widespread bankruptc...,$700 billion US economic bailout,conclusion
3,1,"""Job security: Safeguarding jobs across the ec...",$700 billion US economic bailout,premise
4,2,$700b bailout is generally well designed to so...,$700 billion US economic bailout,conclusion
5,2,Treasury Secretary Henry Paulson summarized th...,$700 billion US economic bailout,premise
6,3,$700b bailout offers buyer for frozen mortgage...,$700 billion US economic bailout,conclusion
7,3,"""Paulson Plan provides a plan Just as optimism...",$700 billion US economic bailout,premise
8,3,Allowing such a trend to gain strength is our ...,$700 billion US economic bailout,premise
9,4,Most economists support the $700b US economic ...,$700 billion US economic bailout,conclusion


### Drop duplicates

In [4]:
df =  df.drop_duplicates(subset="content",keep="first")
df

,argument_id,content,topic,unitType
0,0,Not passing $700b bailout risks sending econom...,$700 billion US economic bailout,conclusion
1,0,U.S. Treasury Secretary Henry Paulson summariz...,$700 billion US economic bailout,premise
2,1,$700b bailout helps avoid widespread bankruptc...,$700 billion US economic bailout,conclusion
3,1,"""Job security: Safeguarding jobs across the ec...",$700 billion US economic bailout,premise
4,2,$700b bailout is generally well designed to so...,$700 billion US economic bailout,conclusion
5,2,Treasury Secretary Henry Paulson summarized th...,$700 billion US economic bailout,premise
6,3,$700b bailout offers buyer for frozen mortgage...,$700 billion US economic bailout,conclusion
7,3,"""Paulson Plan provides a plan Just as optimism...",$700 billion US economic bailout,premise
8,3,Allowing such a trend to gain strength is our ...,$700 billion US economic bailout,premise
9,4,Most economists support the $700b US economic ...,$700 billion US economic bailout,conclusion


### split by topic

In [18]:
topics = df.topic.unique()
n = len(topics)
n

460

In [19]:
n_t = math.floor(n*0.8)
n_t

368

In [21]:
topics_tr = topics[:n_t]
topics_tr

array(['$700 billion US economic bailout', '2009 US economic stimulus',
       '2010 US bank tax', '4-team college football playoff',
       '700 mile US Mexico border fence',
       'Abolishing intellectual property rights',
       'Abolition of art subsidies', 'Abolition of nuclear weapons',
       'Abortion', 'Abstinence-only vs. comprehensive sex education',
       'Adult male circumcision', 'Advances in science present a threat',
       'Advertising for tobacco products should be banned',
       'Affirmative action', 'Aid for persecuted churches', 'AIG bonuses',
       'Airport security profiling', 'Alcohol ban', 'Algae biofuel',
       'Ammonia fuel', 'Animal rights', 'Animal testing',
       'Anorexics, force-feeding of',
       'Are degrees eseential to success in life?',
       'Are schools that are not current on technology limiting their students?',
       'Arizona illegal immigration law', 'Armed pilots in airplanes',
       'Armed police', 'Arranged marriage', 'Artificial 

In [22]:
topics_te = topics[n_t:]
topics_te

array(['Should aboriginals be treated differently under the law?',
       'Should Catholic priests be allowed to marry?',
       'Should everyone in the world speak the same language?',
       'Should governments bailout journalism?',
       'Should governments use open source software?',
       'Should school funding be based on academic achievement?',
       'Should Japan remove limitations on its military?',
       'Should parents raise children without TV?',
       'Should parents trick their kids to believe in Santa Claus?',
       'Should plastic shopping bags be banned?',
       'Should the minimum age of candidacy for political office be 18?',
       'Should the world continue to prosecute Nazi war criminals?',
       'Should Tony Blaire be made the first president of the EU?',
       'Should Turkey join the EU?', 'Should war be televised?',
       'Single-payer universal health care', 'Single-sex schools',
       'Smoking', 'Socialism', 'Social networking sites', 'Solar energy

In [27]:
df_tr = df[df['topic'].isin(topics_tr)]
df_tr

,argument_id,content,topic,unitType
0,0,Not passing $700b bailout risks sending econom...,$700 billion US economic bailout,conclusion
1,0,U.S. Treasury Secretary Henry Paulson summariz...,$700 billion US economic bailout,premise
2,1,$700b bailout helps avoid widespread bankruptc...,$700 billion US economic bailout,conclusion
3,1,"""Job security: Safeguarding jobs across the ec...",$700 billion US economic bailout,premise
4,2,$700b bailout is generally well designed to so...,$700 billion US economic bailout,conclusion
5,2,Treasury Secretary Henry Paulson summarized th...,$700 billion US economic bailout,premise
6,3,$700b bailout offers buyer for frozen mortgage...,$700 billion US economic bailout,conclusion
7,3,"""Paulson Plan provides a plan Just as optimism...",$700 billion US economic bailout,premise
8,3,Allowing such a trend to gain strength is our ...,$700 billion US economic bailout,premise
9,4,Most economists support the $700b US economic ...,$700 billion US economic bailout,conclusion


In [28]:
df_te = df[df['topic'].isin(topics_te)]
df_te

,argument_id,content,topic,unitType
39686,11565,Aboriginals deserve differential treatment as ...,Should aboriginals be treated differently unde...,conclusion
39687,11565,"In almost all instances, aboriginals and settl...",Should aboriginals be treated differently unde...,premise
39688,11565,Those treaties were violated by the settlers i...,Should aboriginals be treated differently unde...,premise
39689,11565,To protect those historical agreements and to ...,Should aboriginals be treated differently unde...,premise
39690,11565,"In Australia and Canada, the courts have held ...",Should aboriginals be treated differently unde...,premise
39691,11565,"Similarly, in Canada, aboriginal groups can se...",Should aboriginals be treated differently unde...,premise
39692,11566,The treaties were made by a colonial power in ...,Should aboriginals be treated differently unde...,conclusion
39693,11566,"Further, those rights were violated by settler...",Should aboriginals be treated differently unde...,premise
39694,11566,"Today’s democratic governments, having to deal...",Should aboriginals be treated differently unde...,premise
39695,11566,Land title claims do not just affect farming l...,Should aboriginals be treated differently unde...,premise


In [30]:
df_tr.to_json("../data/debatepedia/k-fold/set1/debatepedia-preprocessed-train.json", orient = "records")
df_te.to_json("../data/debatepedia/k-fold/set1/debatepedia-preprocessed-test.json", orient = "records")